In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkSession available as 'spark'.


In [2]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local [*]").getOrCreate()

In [3]:
graphPath = "/data/graphDFSample"

In [4]:
from pyspark.sql import functions as f
from pyspark.sql import Window

reversedGraph = sparkSession.read.parquet(graphPath) \
    .withColumn('friend', f.explode('friends')) \
    .groupBy('friend') \
    .agg(f.collect_list('user').alias('users')) \
    .withColumn('users_size', f.size('users')) \

In [5]:
sortedUsers = reversedGraph.select('friend', f.sort_array('users', asc=True).alias('users')) \
    .where("users_size > 1")

In [6]:
sortedUsers.printSchema()
sortedUsers.take(5)

root
 |-- friend: integer (nullable = true)
 |-- users: array (nullable = true)
 |    |-- element: integer (containsNull = true)



[Row(friend=148, users=[3195315, 14631101, 14957568, 65051219]),
 Row(friend=36538, users=[5506394, 6170161, 6846874, 15305594, 17133226, 19016678, 20686207, 20689187, 22582764, 23057528, 27967558, 28135661, 29050446, 31111067, 35731241, 36375468, 38837335, 40003405, 41101961, 41660921, 42973992, 43548989, 44996025, 51969555, 53947972, 57354452, 60011356, 63305254, 63987222, 64553898, 64583713, 65275157]),
 Row(friend=41751, users=[41811068, 60873111]),
 Row(friend=49331, users=[45058971, 58571716]),
 Row(friend=73470, users=[37445156, 49852791])]

In [7]:
sample = sortedUsers.sample(False, 0.00001)
sample.collect()

[Row(friend=41101436, users=[1059916, 2964434, 3042643, 6910864, 7567747, 9516374, 9587570, 19581766, 22024551, 23333155, 29990928, 33198759, 34161415, 35237648, 40442730, 50342783, 50969081, 55582051, 56499329, 58904084, 58968470, 63383096]),
 Row(friend=3178015, users=[41302247, 43792154]),
 Row(friend=10580502, users=[17796133, 19177997, 23140914, 32904168]),
 Row(friend=27471514, users=[39441759, 64823125]),
 Row(friend=52946228, users=[1713532, 42382154, 44385745, 54979070, 60182932]),
 Row(friend=22018979, users=[16851968, 48286812, 54979070]),
 Row(friend=21634931, users=[13585170, 22312953]),
 Row(friend=29710866, users=[31598959, 47356790, 49336362]),
 Row(friend=9738979, users=[20913342, 23020074]),
 Row(friend=44301430, users=[4241591, 6802654, 17265340, 19388348, 23803624, 29167651, 29793337, 30488705, 33793439, 37502021, 40312256, 42994572, 49405386, 53481502, 61979700]),
 Row(friend=61788306, users=[1245641, 6297261, 23644106, 40612643, 41913940, 47874744]),
 Row(friend=5

In [8]:
def genUserPair(users):
    
    pairs = []
    n = len(users)
    
    for i in range(n):
        for j in range(i+1, n):
            pair = [users[i], users[j]]
            pairs.append(pair)
            
    return pairs

In [9]:
genUserPair([3195315, 14631101, 14957568, 65051219])

[[3195315, 14631101],
 [3195315, 14957568],
 [3195315, 65051219],
 [14631101, 14957568],
 [14631101, 65051219],
 [14957568, 65051219]]

In [10]:
from pyspark.sql import types as t
genUserPair_udf = f.udf(genUserPair, t.ArrayType(t.ArrayType(t.IntegerType())))

In [11]:
userPairs = sortedUsers.select('friend', genUserPair_udf('users').alias('pairs').alias('pairs')) \
    .withColumn('pair', f.explode('pairs')) \
    .withColumn('user1', f.col('pair')[0]) \
    .withColumn('user2', f.col('pair')[1]) \
    .select('friend', 'user1', 'user2')

In [12]:
userPairs.printSchema()
userPairs.take(5)

root
 |-- friend: integer (nullable = true)
 |-- user1: integer (nullable = true)
 |-- user2: integer (nullable = true)



[Row(friend=148, user1=3195315, user2=14631101),
 Row(friend=148, user1=3195315, user2=14957568),
 Row(friend=148, user1=3195315, user2=65051219),
 Row(friend=148, user1=14631101, user2=14957568),
 Row(friend=148, user1=14631101, user2=65051219)]

In [13]:
mutualFriends = userPairs.groupBy('user1', 'user2') \
    .agg(f.count('friend').alias('mutual_friends'))

In [ ]:
mutualFriends.printSchema()
mutualFriends.take(5)

root
 |-- user1: integer (nullable = true)
 |-- user2: integer (nullable = true)
 |-- mutual_friends: long (nullable = false)



[Row(user1=19016678, user2=27967558, mutual_friends=25),
 Row(user1=35731241, user2=63987222, mutual_friends=1272),
 Row(user1=38837335, user2=60011356, mutual_friends=2),
 Row(user1=40003405, user2=41101961, mutual_friends=26),
 Row(user1=3274722, user2=37391049, mutual_friends=100)]

In [ ]:
window = Window.orderBy(f.col('mutual_friends').desc())
    
top50 = mutualFriends.withColumn('row_number', f.row_number().over(window)) \
    .filter('row_number < 50') \
    .select('mutual_friends', 'user1', 'user2') \
    .orderBy(f.col('mutual_friends').desc()) \
    .collect()

In [ ]:
for val in top50:
    print '%s %s %s' % val

3206 27967558 42973992
3130 20158643 42973992
3066 22582764 42973992
3044 21864412 51640390
3021 17139850 51640390
3010 14985079 51640390
2970 17139850 21864412
2913 20158643 27967558
2903 22280814 51151280
2870 23848749 51640390
2855 20158643 22582764
2849 20158643 44996025
2846 22280814 42973992
2784 21864412 23848749
2779 31964081 51640390
2776 39205988 51640390
2754 17139850 23848749
2749 22582764 27967558
2728 50561859 51640390
2724 15485897 51640390
2700 28135661 42973992
2655 22280814 27967558
2653 42973992 43548989
2639 26755857 51640390
2621 14635589 51640390
2608 15485897 17139850
2606 17139850 26755857
2601 21864412 39205988
2600 8406745 51640390
2599 37735419 51640390
2597 20158643 28135661
2585 21864412 31964081
2585 40003405 42973992
2581 27967558 43548989
2579 23848749 31964081
2578 15485897 21864412
2578 27967558 28135661
2577 42973992 64755069
2574 51151280 57077210
2573 20158643 43548989
2566 21864412 26755857
2564 22280814 64755069
2561 42973992 44996025
2556 1713985